# Reporte de MLT de Barcelona Telco

## Problema ha solucionar


## Data

In [ ]:
!apt-get install swig -y
!pip install Cython numpy
!pip install auto-sklearn

In [2]:
import time
import matplotlib.pyplot as plt
import seaborn as sns  
import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/sergiakalorth/MachineLearning-CIM-UPC/master/Final%20Project/datasets/Wine/winequality-red.csv',header=0)
print(df.shape)

(1599, 12)


In [3]:
df.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Data Exploration

In [5]:
df.isnull().values.any()

False

In [6]:
df.dtypes

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

In [ ]:
df['quality'].value_counts()

In [ ]:
sns.pairplot(df, hue="quality", palette="husl")

## Algoritmo de Machine Learning

In [ ]:
Y = df['quality']
data = df.drop(['quality'],axis=1)
X = data.values

X, Y = shuffle(X,Y)
x_train = []
y_train = []
x_test = []
y_test = []

x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.9,random_state=30)

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

print(x_train.shape)
print(x_test.shape)

In [ ]:
#import autosklearn.classification
#import sklearn.model_selection
#import sklearn.datasets
#import sklearn.metrics

#automl = autosklearn.classification.AutoSklearnClassifier(
#          time_left_for_this_task=1200, 
#          per_run_time_limit=300,
#          )

# train model(s)
#automl.fit(x_train, y_train)


## Resultados

In [ ]:
# evaluate
#y_hat = automl.predict(x_test)
#test_acc = sklearn.metrics.accuracy_score(y_test, y_hat)

#print("Test Accuracy score {0}".format(test_acc))
#print(automl.sprint_statistics())
#print(automl.show_models())

 ## Discusión y Conclusiones

## Linias de desarrollo futuro

### Annexo 1


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import joblib
filename = '/content/gdrive/My Drive/Colab Notebooks/Projecte Final IML/Modelos/model_Wine_AutoIML.pkl'
joblib.dump(automl, filename)

### Annexo 2

In [ ]:
print("Los valores de {} son max: {} i min: {}".format(df.columns[0], df[df.columns[0]].max(), df[df.columns[0]].min()))
print("Los valores de {} son max: {} i min: {}".format(df.columns[1], df[df.columns[1]].max(), df[df.columns[1]].min()))
print("Los valores de {} son max: {} i min: {}".format(df.columns[2], df[df.columns[2]].max(), df[df.columns[2]].min()))
print("Los valores de {} son max: {} i min: {}".format(df.columns[3], df[df.columns[3]].max(), df[df.columns[3]].min()))
print("Los valores de {} son max: {} i min: {}".format(df.columns[4], df[df.columns[4]].max(), df[df.columns[4]].min()))
print("Los valores de {} son max: {} i min: {}".format(df.columns[5], df[df.columns[5]].max(), df[df.columns[5]].min()))
print("Los valores de {} son max: {} i min: {}".format(df.columns[6], df[df.columns[6]].max(), df[df.columns[6]].min()))
print("Los valores de {} son max: {} i min: {}".format(df.columns[7], df[df.columns[7]].max(), df[df.columns[7]].min()))
print("Los valores de {} son max: {} i min: {}".format(df.columns[8], df[df.columns[8]].max(), df[df.columns[8]].min()))
print("Los valores de {} son max: {} i min: {}".format(df.columns[9], df[df.columns[9]].max(), df[df.columns[9]].min()))
print("Los valores de {} son max: {} i min: {}".format(df.columns[10], df[df.columns[10]].max(), df[df.columns[10]].min()))

In [ ]:
fixedAcidity = 5
volatileAcidity = 0.12
citricAcid = 0.25
residualSugar = 2
chlorides = 0.3
freeSulfurDioxide = 25
totalSulfurDioxide = 10
density = 1
pH = 3.3
sulphates = 0.62
alcohol = 10.5
datosEjemplo = np.array([fixedAcidity,volatileAcidity,citricAcid,residualSugar,chlorides,freeSulfurDioxide,totalSulfurDioxide,density,pH,sulphates,alcohol])

In [ ]:
datosNuevos = datosEjemplo.reshape(1,-1)
automl_wine = joblib.load('/content/gdrive/My Drive/Colab Notebooks/Projecte Final IML/Modelos/model_Wine_AutoIML.pkl')
score = automl_wine.score(x_test,y_test)
print("Test score: {0:.2f} %".format(100 * score))
#print(automl_wine.sprint_statistics())
#print(automl_wine.show_models())
qualityEsperada = automl_wine.predict(datosNuevos)
print(qualityEsperada)

In [ ]:
datosNuevos = datosFerran.reshape(1,-1)
qualityEsperada = RFC.predict(datosNuevos)
score = RFC.score(x_test,y_test)
print("Test score: {0:.2f} %".format(100 * score))
print(qualityEsperada)

imlpython.westeurope.cloudapp.azure.com:1880/ui